In [1]:
print('hello')

hello


In [20]:
!pip uninstall -y -q bigframes cesium tsfresh mlxtend imbalanced-learn \
                      google-cloud-automl pandas-gbq plotnine dopamine-rl || true

!pip install -q \
    azure-ai-inference "langchain>=0.2.3,<0.3" \
    datasets spacy tiktoken python-dotenv rich \
    sentence-transformers

!python -m spacy download en_core_web_sm --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 96.5 MB/s eta 0:00:00:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do 

In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Cell 2
import os, json, pandas as pd
from datasets import load_dataset

# ---- GitHub GPT-4.1 wrapper via Azure AI Inference ----
import os, json, pandas as pd
from datasets import load_dataset
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from langchain.llms.base import LLM

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("github-openai")



# token = os.getenv("github-openai")
assert token, "Add GITHUB_TOKEN under Secrets!"

client = ChatCompletionsClient(
    endpoint="https://models.github.ai/inference",
    credential=AzureKeyCredential(token),
)
MODEL_ID = "openai/gpt-4.1"

class GithubGPT4(LLM):
    def _call(self, prompt, **kwargs):
        resp = client.complete(
            model=MODEL_ID,
            messages=[SystemMessage("Return ONLY valid JSON."), UserMessage(prompt)],
            temperature=0.0, top_p=1.0,
        )
        return resp.choices[0].message.content
    @property
    def _llm_type(self): return "github-gpt4"

llm = GithubGPT4()



In [22]:
CURRENT_YEAR = 2025

resume_sys = f"""
You are “ResumeFeatureExtractor v2”.
Return ONLY JSON validating this schema:

{{{{
  "total_years_exp":        "int",     // current_year - earliest year found
  "current_role_tenure":    "int",     // current_year - most recent start-year
  "education":              "str",     // highest degree or "Unknown"
  "seniority":              "str",     // Junior | Mid | Senior | Lead | Director | VP | C-Level
  "skills":                 "string[]",// up to 30 distinct keywords
  "certifications":         "string[]",// e.g. CPA, CCNA
  "locations":              "string[]",// city + state or country codes
  "highlight_count":        "int",     // bullets under any 'Highlights' section
  "longest_tenure":         "int"      // longest continuous job stint, yrs
}}}}

Extraction rules:
• Use {CURRENT_YEAR} as ‘now’.
• If no explicit “Skills:” section, infer from bullet points & job duties.
• Return skills / cert names in Title Case, no duplicates.
• Always include every key (use 0, "", [] if absent).
• **ONLY JSON** — no markdown, no prose."""

jd_sys = """
You are “JDFeatureExtractor v2”.
Return ONLY JSON validating this schema:

{{{
  "required_degree":       "str",
  "required_skills":       "string[]",   // up to 30
  "desired_experience":    "str",        // Junior | Mid | Senior | Lead
  "industry":              "str",
  "location":              "str",
  "security_requirement":  "bool",
  "expense_bullet_count":  "int",
  "software_mentions":     "string[]"
}}}

Rules:
• Treat phrases like “B.S.” as "Bachelor", “MSc” → "Master".
• security_requirement = true if US citizenship / clearance / ITAR required.
• expense_bullet_count = bullets mentioning reimbursement, corporate card, etc.
• software_mentions = distinct product names (max 15).
• Always include every key.
• **ONLY JSON**.
"""


In [23]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer("all-MiniLM-L6-v2")   # 384-d vectors

def ask_json(sys_prompt: str, text: str, tries: int = 3) -> dict:
    chunk = text[:6500]
    for _ in range(tries):
        try:
            msgs = [SystemMessage(sys_prompt), UserMessage(chunk)]
            return json.loads(llm.generate(msgs).generations[0][0].text)
        except Exception:
            chunk = "Return ONLY valid JSON.\n\n" + chunk
    return {}

def extract_resume(txt): return ask_json(resume_sys, txt)
def extract_jd(txt):     return ask_json(jd_sys, txt)

def embed(text: str):
    vec = embedder.encode([text])[0]
    # store first 8 dims for compact display
    return [round(float(x),4) for x in vec[:8]]


2025-05-25 05:09:42.723450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748149783.163015      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748149783.296973      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
data = load_dataset("cnamuangtoun/resume-job-description-fit", split="train").to_pandas()
sample = data.head(2)

rows = []
for _, row in sample.iterrows():
    res_js = extract_resume(row["resume_text"])
    jd_js  = extract_jd(row["job_description_text"])
    
    rows.append({
        "label": row["label"],
        **{f"res_{k}": v for k,v in res_js.items()},
        **{f"jd_{k}":  v for k,v in jd_js.items()},
        "res_summary_emb": embed(row["resume_text"].split("Highlights")[0][:512]),
        "jd_emb":           embed(row["job_description_text"][:512]),
    })

pd.set_option("display.max_colwidth", 140)
pd.DataFrame(rows)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,label,res_total_years_exp,res_current_role_tenure,res_education,res_seniority,res_skills,res_certifications,res_locations,res_highlight_count,res_longest_tenure,jd_required_degree,jd_required_skills,jd_desired_experience,jd_industry,jd_location,jd_security_requirement,jd_expense_bullet_count,jd_software_mentions,res_summary_emb,jd_emb
0,No Fit,12,2,Master Of Business Administration,Senior,"[Project Management, Agile, Scrum, Team Leadership, Stakeholder Management, Budgeting, Risk Management, Process Improvement, Business An...","[Pmp, Certified Scrum Master]","[New York, NY, San Francisco, CA]",7,5,Bachelor,"[Python, data analysis, machine learning, communication, problem solving, teamwork, statistics, data visualization]",Mid,Technology,"San Francisco, CA",False,1,"[Python, Tableau, Excel, SQL]","[-0.0424, 0.0165, 0.029, -0.0098, -0.0332, 0.0058, 0.0479, 0.0629]","[-0.0566, -0.0766, 0.0086, 0.0201, -0.0172, 0.0292, -0.0049, 0.0474]"
1,No Fit,12,3,Master Of Business Administration,Senior,"[Project Management, Agile, Scrum, Team Leadership, Stakeholder Management, Budgeting, Risk Management, Process Improvement, Business An...","[Pmp, Scrum Master]","[New York, NY, Boston, MA]",0,5,Bachelor,"[Java, Spring Boot, REST APIs, SQL, Agile methodologies, Unit testing, Version control, Problem solving, Communication, Teamwork]",Mid,Information Technology,"Austin, TX",False,0,"[Java, Spring Boot, Git]","[-0.0541, 0.0182, 0.0595, 0.1083, -0.0778, -0.0407, 0.0365, 0.032]","[-0.0033, 0.115, -0.0025, -0.0097, 0.0598, 0.0151, 0.0032, 0.0147]"
